In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법_with_markdown.docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

194

In [4]:
document_list[46]

Document(metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40%) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40%) |\n\n| 5억

In [5]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
#     model="solar-embedding-1-large-passage"
)

In [6]:
len(embedding.embed_query("소득세법"))

4096

In [7]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index-markdown"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# 업로드한 벡터db를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 6.44 s
Wall time: 46.3 s


# 2. 답변 생성 전 retrieval 확인

In [8]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [10]:
retrieved_docs[0]

Document(id='b0875c5f-d9e0-4cc3-a16c-e1bf3b32764d', metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40%) |\n\n| 3억원 초과 5억원 이

In [11]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [12]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [14]:
retrived_docs[0]

Document(id='b0875c5f-d9e0-4cc3-a16c-e1bf3b32764d', metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40%) |\n\n| 3억원 초과 5억원 이

# 3. 답변 생성

In [15]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {
            "role":"user",
            "content": f"""- [context]를 참고해서 사용자의 질문에 5줄이내로 답변해 주세요
- [context] : {retrieved_doc}
- 질문 : {query}"""
        }
    ],
    temperature=0.2
)

In [16]:
print(response.choices[0].message.content)

연봉이 5천만원인 직장인의 종합소득세는 다음과 같이 계산됩니다. 5천만원의 경우, 과세표준은 5천만원 초과 8,800만원 이하에 해당하므로, 세액은 624만원 + (5천만원 초과 금액인 0원을 24%)로 계산됩니다. 따라서, 소득세는 624만원입니다.


In [17]:
from openai import OpenAI
import os
# upstage의 과금정책 : https://www.upstage.ai/pricing/api
client = OpenAI(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1"
)
response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {"role":"system", "content":"당신은 최고의 한국 소득세법 전문가입니다"},
        {
            "role":"user",
            "content": f"""- [context]를 참고해서 사용자의 질문에 10줄이내로 답변해 주세요
- [context] : {retrieved_doc}
- 질문 : {query}"""
        }
    ],
    temperature=0.2
)

In [18]:
print(response.choices[0].message.content)

연봉 5,000만원 직장인의 소득세는 다음과 같이 계산됩니다(2023년 기준, 근로소득공제 및 기본공제 적용 후):  

1. **근로소득공제**: 5,000만원 × 15% - 90만원 = **660만원**  
   (공제 후 과세표준: 5,000만원 - 660만원 = **4,340만원**)  

2. **종합소득세 계산**:  
   - 1,400만원 이하: 1,400만원 × 6% = **84만원**  
   - 1,400만원 초과 4,340만원: (4,340만원 - 1,400만원) × 15% = **438만원**  
   - **산출세액**: 84만원 + 438만원 = **522만원**  

3. **세액공제 적용** 후 최종 납부세액은 약 **400~450만원** 수준입니다(자녀·연금 등 추가 공제 유무에 따라 변동).  

※ 정확한 금액은 추가 공제 항목(의료비, 보험료 등)과 지방소득세(10%)를 고려해야 합니다.


In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4.1-nano")

In [20]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [21]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [22]:
ai_message = llm.invoke(prompt)

In [23]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 **근로소득공제**를 적용한 후 과세표준을 산출하고, 해당 과세표준에 대해 **누진세율**을 적용해야 합니다. 다만, 제공된 [context]에는 근로소득공제율이 명시되어 있지 않아 정확한 계산이 어렵습니다. 일반적인 한국 소득세법을 기준으로 예시를 들어 설명드리겠습니다.

---

### 1. **근로소득공제 적용 (추정)**
한국의 근로소득공제율은 다음과 같습니다(2023년 기준, 예시):
- **총급여액 5천만원**인 경우, 근로소득공제는 약 **1,225만원**입니다.  
  (※ 실제 공제액은 근로소득공제율표에 따라 상세히 계산되며, 2천만원 한도 적용)

### 2. **과세표준 산정**
- **총급여액**: 5,000만원  
- **근로소득공제**: 1,225만원  
- **과세표준**: 5,000만원 - 1,225만원 = **3,775만원**

---

### 3. **종합소득세 산출세액 계산**
제55조 세율표를 적용하여 과세표준 3,775만원에 대한 세금을 계산합니다:
- **1,400만원 이하**: 1,400만원 × 6% = **84만원**  
- **1,400만원 초과 3,775만원 이하**: (3,775만원 - 1,400만원) × 15% = 2,375만원 × 15% = **356.25만원**  
- **총 산출세액**: 84만원 + 356.25만원 = **440.25만원**

---

### 4. **세액공제 적용 (예시)**
- **자녀세액공제**, **연금계좌세액공제** 등이 적용될 수 있으나, 추가 정보가 없어 생략합니다.  
- 실제 세액은 공제 항목에 따라 감소할 수 있습니다.

---

### 5. **최종 소득세 (예시)**
- **산출세액**: 약 **440만원** (공제 전)  
- **실제 납부세액**: 공제 적용 후 약 **350~420만원** 범위 (가족 상황, 추가 공제 항목에 따라 변동)

---

### 📌 **중요 참고 사항**
- 제공된 [context]에

# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase을 활용한 RAG구현(UpstageEmbedding) 참조

In [26]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model = "gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")

embedding = UpstageEmbeddings(model="solar-embedding-1-large")

# 2. # 업로드한 벡터db를 가져올 때
vectorstore = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# 4. 프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다.
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return  "\n\n---\n\n".join([doc.page_content for doc in documents])

In [27]:
# 6. RAG 체인 구성 (LCEL 방식)
from langchain_core.runnables import RunnablePassthrough # {"query":"~"} => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough() # 질문 그대로 전달
    }
    | prompt # prompt에 context와 query 변수 주입
    | llm    
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'연봉 5천만원의 근로소득세 계산은 다음과 같습니다.  \n1. **근로소득공제 적용 후 과세표준**: 5,000만원 - (5,000만원 × 15% + 108만원) = 3,922만원  \n2. **종합소득세 계산**: 84만원 + (1,400만원 초과분 2,522만원 × 15%) = **1,221만원**  \n3. **근로소득세액공제 적용 후 최종 세액**: 1,221만원 - (근로소득세액공제액 약 74만원) ≈ **1,147만원**  \n(정확한 금액은 추가 공제 항목에 따라 달라질 수 있음)  \n\n※ 단, 실제 세액에는 건강보험료·국민연금 등 공제 및 지방소득세(10%)가 추가로 적용됩니다.'

# 5. 키워드 사전 활용

In [28]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [30]:
vec1 = embedding.embed_query("총급여")
vec2 = embedding.embed_query("연봉")
print("총급여와 연봉의 유사도 : ",cosine_similarity(vec1,vec2))

총급여와 연봉의 유사도 :  0.6872200519738804


In [31]:
vec1 = embedding.embed_query("종합소득")
vec2 = embedding.embed_query("연봉")
print("종합소득와 연봉의 유사도 : ",cosine_similarity(vec1,vec2))

종합소득와 연봉의 유사도 :  0.56632913237249


In [32]:
vec1 = embedding.embed_query("직장인")
vec2 = embedding.embed_query("거주자")
print("직장인와 거주자의 유사도 : ",cosine_similarity(vec1,vec2))

직장인와 거주자의 유사도 :  0.540122403716264


In [33]:
vec1 = embedding.embed_query("5천만원")
vec2 = embedding.embed_query("5,000만원")
print("5천만원와 5,000만원의 유사도 : ",cosine_similarity(vec1,vec2))

5천만원와 5,000만원의 유사도 :  0.9122554959328768


In [34]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = vectorstore.similarity_search(query,k=2)
retrieved_docs

[Document(id='b0875c5f-d9e0-4cc3-a16c-e1bf3b32764d', metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40%) |\n\n| 3억원 초과 5억원 

In [35]:
query = "연봉 5000만원인 거주자의 소득세는 얼마인가요?"
retrieved_docs = vectorstore.similarity_search(query,k=2)
retrieved_docs

[Document(id='b0875c5f-d9e0-4cc3-a16c-e1bf3b32764d', metadata={'source': 'data/소득세법_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n\n\n\n\n| 종합소득 과세표준 | 세율 |\n\n| 종합소득 과세표준 | 세율 |\n\n|---|---|\n\n|---|---|\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 이하 | 과세표준의 6% |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 1,400만원 초과 5,000만원 이하 | 84만원 + (1,400만원을 초과하는 금액의 15%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 5,000만원 초과 8,800만원 이하 | 624만원 + (5,000만원을 초과하는 금액의 24%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 8,800만원 초과 1억5천만원 이하 | 1,536만원 + (8,800만원을 초과하는 금액의 35%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 1억5천만원 초과 3억원 이하 | 3,706만원 + (1억5천만원을 초과하는 금액의 38%) |\n\n| 3억원 초과 5억원 이하 | 9,406만원 + (3억원을 초과하는 금액의 40%) |\n\n| 3억원 초과 5억원 

In [56]:
# 사람을 나타내는 표현 -> 거주자로 변경/ 5천만원->5000만원 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
llm = ChatOpenAI(model="gpt-4o-mini")
dictionary = ["사람을 나타내는 표현 -> 거주자"]
prompt = ChatPromptTemplate.from_template(f"""사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요. 만약 변경할 필요가 없을 경우, 사용자의 질문을 변경하지 않아도 됩니다.
질문만 리턴해주세요
사전 : {dictionary}
질문 : {{question}}""")

In [57]:
parser = output_parser = StrOutputParser()
parser.invoke(llm.invoke(prompt.invoke({"question":"소득이 높은 남자가 있습니다"})))

'소득이 높은 거주자가 있습니다.'

In [58]:
# langchain
dictionary_chain = prompt | llm | StrOutputParser()
dictionary_chain.invoke({"question":"소득이 높은 남자가 5천만원 가지고 있습니다."})

'소득이 높은 거주자가 5천만원 가지고 있습니다.'

In [59]:
dictionary_chain.invoke("연봉 5천만원인 직장인의 소득세는 얼마예요?")

'연봉 5천만원인 거주자의 소득세는 얼마예요?'

In [60]:
#rag_chain.invoke("연봉 5천만원인 직장인의 소득세는 얼마예요?")
final_chain = dictionary_chain | rag_chain

In [61]:
final_chain.invoke("연봉 5천만원인 직장인의 소득세는 얼마예요?")

'연봉 5천만원에 대한 소득세 계산은 **근로소득공제, 과세표준, 세율, 세액공제** 등을 모두 고려해야 하므로 단순 연봉만으로는 정확한 금액을 알 수 없습니다. 제공된 문맥에서는 근로소득공제율 또는 추가 공제 항목이 명시되지 않아 정확한 계산이 불가능합니다.  \n\n간소화된 예시로, 근로소득공제 후 과세표준이 **5,000만원 이하**라면 세액은 **624만원**이지만, 실제 세액에는 추가 공제가 적용될 수 있습니다.  \n\n정확한 계산을 위해서는 총급여액, 공제 항목(보험료, 의료비 등), 세액감면 등을 종합적으로 검토해야 합니다.'

In [ ]:
# 입력 {"question":"질문"} -> dictionary_chain이 질문 멘트를 개선
# -> 개선된 질문을 rag_chain에 전달 -> retriever가 관련 문서를 검색 -> format_documents
# -> 완성된 prompt를 llm에 전달하여 답변 생성 -> 문자만 추출해서 최종 답변